<a href="https://colab.research.google.com/github/EduGame3/Experiment/blob/master/T2_f_Opciones_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 447 (delta 105), reused 0 (delta 0), pack-reused 311
Receiving objects: 100% (447/447), 1.53 MiB | 18.01 MiB/s, done.
Resolving deltas: 100% (288/288), done.


In [0]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
# require(data.table)
install.packages("data.table")
library(data.table)

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/quantmod_0.4.17.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 152204 bytes (148 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [0]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}


diagm=function(x)		#función para diagonalizar una matriz en una de mayor dimensión
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  if(m01>=n01 )
  {
    res=matrix(0,m01,m01)
    for (i in 1:(m01/n01))
    {
      z=(i-1)*n01
      for(j in 1:n01)
      {
        for(k in 1:n01)
        {
          res[z+k,z+j]=x[k,z+j]
        }
      }
    }
    res
  }
  else
  {
    "no es matriz cuadrada"
  }
}

# Definición de parámetros para valorar:
### Opciones: Cargamos parámetros para valoración:

In [0]:
%%R
fval=as.Date("20200228",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.95 #Nivel de confianza para obtener estimaciones de riesgo
nh=3660 #número de días de historia
yext=1 #si se usa la historia de internet o fija

#cargamos los factores de riesgo. Son dos curvas y una superficie de volatilidad.
btasadesc_oir="RiesgosFinancieros/2020-1/Insumos/tasa_TIIE_SW_OP.txt"
btasaspot_oir="RiesgosFinancieros/2020-1/Insumos/tasa_DIRS_SW_OP.txt"
bvolspot_oir="RiesgosFinancieros/2020-1/Insumos/tvoltiie_opc.txt"
plazos_oir=cbind( 1700, 700) #T-t, la temporalidad son días 
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor)
K_oir=cbind( 0.094, 0.089)
contratos_oir=cbind(1000, 500)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa 0 si es simple

# Carga de datos

In [0]:
%%R
#CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
  #carga de datos
  #carga de rho ( tasa de descuento )	
  data1_opc<-read.table(btasadesc_oir) 
  n_opc<- row(data1_opc)
  m1_orig_oir=ncol(data1_opc)
  x1_orig_oir=data.table(mutate(data1_opc[2:n_opc,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1_opc[1,2:m1_orig_oir])
  
  #carga de tasas spot (cupones)
  data2_opc<-read.table(btasaspot_oir)
  n_opc<-nrow(data2_opc)
  m2_orig_oir=ncol(data2_opc)
  x2_orig_oir=data.table(mutate(data2_opc[2:n_opc,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2_opc[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3_opc<-read.table(bvolspot_oir)
  n_opc<-nrow(data3_opc)
  m3_orig_oir=ncol(data3_opc)
  x3_orig_oir=data.table(mutate(data3_opc[2:n_opc,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3_opc[1,2:m3_orig_oir])
  print(n_opc)

[1] 256


# Integración de Insumos.
Nos aseguramos de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. Posteriormente, filtramos cada histórico factor de riesgo hacia esta intersección de todos.

In [0]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS (intersectamos 3 curvas, correspondientes a los factores de riesgo ) 
  
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(x1_orig_oir[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol

R[write to console]: Error in colnamesInt(x, names(on), check_dups = FALSE) : 
  argument specifying columns specify non existing column(s): cols[1]='Date'
Calls: <Anonymous> ... data.table -> as.Date -> [ -> [.data.table -> colnamesInt




Error in colnamesInt(x, names(on), check_dups = FALSE) : 
  argument specifying columns specify non existing column(s): cols[1]='Date'
Calls: <Anonymous> ... data.table -> as.Date -> [ -> [.data.table -> colnamesInt


In [0]:
%%R
n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol) #Historia de todos los datos
print(n)
#historia de opciones
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)

[1] 1


# Pasos para medir el riesgo
##Delta-Normal
Para todos los instrumentos hay que definir y calcular los siguientes elementos:\
1.Historico de factores de riesgo\
2.Vector de precios actual\
3.Derivada con respecto al tiempo evaluada en el precio actual\
4.Primera derivada con respecto a factores de riesgo evaluada y multiplicada con su precio actual\
## Delta-Gamma
Consideramos los puntos anteriores y :\
5.Segunda derivada con respecto a factores de riesgo evaluada y multiplicada con sus precios actuales (al cuadrado)\
## Delta-CF


##Opciones de tasas de interes

##Función de valoración,derivadas y matriz hessiana

In [0]:
%%R
##opciones de tasa de interés, con inicio el día de la valuación CÁLCULO
#Posición inicial

#interpolación de tasas y volatilidades
m_opc=ncol(plazos_oir)  #m significa el número de contratos de opciones que se van a calcular (en este caso m = 2, cap y floor)
#factores de riesgo:
x1_opc=matrix(0,n_opc,m_opc)    #(tasa de descuento, rho)
x2tc=matrix(0,n_opc,m_opc) #x2 es el factor de riesgo tasa spot, lo dividimos en tasa larga y tasa corta.
x2tl=matrix(0,n_opc,m_opc)
x2_opc=matrix(0,n_opc,m_opc)  # tasa spot 
x3_opc=matrix(0,n_opc,m_opc)  # volatilidad
for (i in 1:(n_opc))
{
  x1_opc[i,]=if(itpl_opc==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl_opc==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl_opc==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3_opc[i,]=if(itpl_opc==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2_opc[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}

x01=x1_opc[1,] #tasas de descuento
x02=x2_opc[1,] #tasas spot
x03=x3_opc[1,] #volatilidades


#procedemos a valorar la opción tdi según los datos de la presentación
 opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
 {
   d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
   d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
   vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
   (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
   #cs_oir=1 #1 si es continua la tasa 0 si es simple
 }

#valoración de opciones tasa de interés
 V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio
 
#DERIVAMOS B&S

#PRIMERA DERIVADA

 dopctintt = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y TIEMPO
 {
   d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
   d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
   vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
   (if(cs_oir==1){(( -(S*dnorm(d1)*vol)/(2*sqrt(t/365))+(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(t/365)*(-1)^cp_oir)*(exp(-vp*t/365)))}else{(( -(S*dnorm(d1)*vol)/(2*sqrt(t/360))+(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(t/360)*(-1)^cp_oir)*(1/(1+vp*t/360)^2))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
 }
 #dopctintt(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

 dopctintS = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y spot
 {
   d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
   d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
   vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
   (if(cs_oir==1){((pnorm(d1*(-1)^cp_oir))*(exp(-vp*t/365)))}else{(pnorm(d1*(-1)^cp_oir)*(1/(1+vp*t/360)))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)- (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2
 }
 #dopctintS(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

 dopctintvol = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y spot (volatilidad)
 {
   d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
   d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
   vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
   (if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
 }
 #dopctintvol(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

 dopctintrho = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y rho (tasa de interés)
 {
   d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
   d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
   vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
   (if(cs_oir==1){-(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(t/365)}else{-(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^2)*(-1)^cp_oir*(t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
 }
 #dopctintrho(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

 #matríz hessiana (cuadrada, de 3x3)
 mgammaopc=function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)	#matriz de segundas derivadas parciales de la call: rho, spot y vol de spot
 {
   d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
   d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
   vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
   c11=(if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(t/365)^2}else{2*(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^3)*(-1)^cp_oir*(t/360)^2})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #rho con respecto a rho
   c21=(if(cs_oir==1){((pnorm(d1*(-1)^cp_oir))*(exp(-vp*t/365)))*(-t/365)}else{(pnorm(d1*(-1)^cp_oir)*(1/(1+vp*t/360)^2))*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)- (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(-t/365)}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^2)*(-1)^cp_oir*(-t/360)})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2*contratos_oir*nominal_oir #spot con respecto a rho
   c31=(if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))*(-t/365)}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360)^2)*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #vol con respecto a rho
   c12=c21 #rho con respecto a spot
   c22=(if(cs_oir==1){(dnorm(d1)/(S*vol*sqrt(t/365)))*(exp(-vp*t/365))}else{(dnorm(d1)/(S*vol*sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir) -2*((if(cs_oir==1){(((pnorm(d1*(-1)^cp_oir))+( S*dnorm(d1*(-1)^cp_oir)-K_oir*dnorm(d2*(-1)^cp_oir)))*(exp(-vp*t/365)))*(-1)^cp_oir*(1/(S*vol*sqrt(t/365)))}else{(((pnorm(d1*(-1)^cp_oir))+( S*dnorm(d1*(-1)^cp_oir)-K_oir*dnorm(d2*(-1)^cp_oir)))*(1/(1+vp*t/360)))*(1/(S*vol*sqrt(t/360)))*(-1)^cp_oir})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2) +2*(if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)^3/(1+S*pr_oir/dct_oir)^3*contratos_oir*nominal_oir #spot spot 
   c32=(if(cs_oir==1){dnorm(d1)*d2/vol*(exp(-vp*t/365))}else{dnorm(d1)*d2/vol*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir) +   (if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2*contratos_oir*nominal_oir # vol spot
   c13=(if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))*(-t/365)}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360)^2)*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #rho vol
   c23=c32 #spot vol
   c33=(if(cs_oir==1){(dnorm(d1)*(S*sqrt(t/365)))*((d1*d2)/vol)*exp(-vp*t/365)}else{(dnorm(d1)*(S*sqrt(t/360)))*((d1*d2)/vol)*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #vol vol
   # outm=matrix(0,3,length(plazos_oir)*3)
   m_opc=length(t)
  rbind(cbind(diagv(c11),diagv(c12),diagv(c13)),cbind(diagv(c21),diagv(c22),diagv(c23)),cbind(diagv(c31),diagv(c32),diagv(c33)))
 }

mgammaopc(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)

 X_oir=cbind(x1_opc,x2_opc,x3_opc)                   #ponemos en un vector los factores de riesgo, en total hay X_oir
 xp_oir=cbind(t(x01),t(x02),t(x03))      #valor actual de los X0
 mu_oir=dopctintt(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir         #derivada con respecto al tiempo
 delta_oir_T=cbind(dopctintrho(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir, dopctintS(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir,dopctintvol(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir)      #Primera derivada (gradiente) evaluado en el punto X0
 deltaT_oir_T=xp_oir*delta_oir_T       #el gradiente multiplicado por el factor de riesgo, por X0
 gammaT_oir=mgammaopc(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir) #la matríz hessiana evaluada en X0
 gammaT_oirT2=gammaT_oir*(t(xp_oir)%*%xp_oir)    #la hessiana multiplicada por X0^2

print(xp_oir)


R[write to console]: Error in approx(nodos1_oir, x1_orig_oir[i, ], plazos_oir, rule = 2) : 
  need at least two non-NA values to interpolate
Calls: <Anonymous> -> <Anonymous> -> withVisible -> approx




Error in approx(nodos1_oir, x1_orig_oir[i, ], plazos_oir, rule = 2) : 
  need at least two non-NA values to interpolate
Calls: <Anonymous> -> <Anonymous> -> withVisible -> approx


# Integración de factores y cálculo de varianza-covarianza

In [0]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
n_if_opc=matrix(0,1,1)      #son opciones, por eso en nrow es 1, las columnas serán el número total de factores de riesgo que hay (línea siguiente)
n_if_opc[1]=ncol(X_oir)     #ncol(X_oir) es 6, que son los faxctores de riesgo

#valor del portafolios
V0_port_opc=cbind(V0_oir)   #valor actual del portafolio
V0T_port_opc=sum(V0_port_opc)

#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port_opc=cbind(X_oir) #Factores de riesgo del portafolios (son 3 por cada contrato)

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port_opc=as.matrix(log(X_port_opc[1:(n_opc-1)]/X_port_opc[2:(n_opc)]))
DeltaX_port_opc[is.nan(DeltaX_port_opc)] <- 0 #quitamos NaN
DeltaX_port_opc[is.na(DeltaX_port_opc)] <- 0 #quitamos Na
DeltaX_port_opc[is.infinite(DeltaX_port_opc)] <- 0 #quitamos Na

print(n_opc)
print(head(X_port_opc))

[1] 256
           [,1]       [,2]       [,3]       [,4]     [,5]      [,6]
[1,] 0.09505543 0.08759649 0.05709562 0.06763294 0.281736 0.2494769


#Medidas de riesgo
## Riesgo Opción Tasa de Interes
Calculemos el VaR a nivel contrato y a nivel contrato factor de riesgo.

In [0]:
%%R
print(m_opc)  #m es el número de opciones
tail(X_port_opc)  
#6 columnas, una por cada factor de riesgo de todo el portafolio. Los primeros 2 son las tasas de descuentos (rho); los segundos 2 son los cupones (tasa spot); los últimos 2 son la tasa de volatilidad.
# Necesitamos una columna de cada tipo por instrumento.

[1] 2
           [,1]       [,2]       [,3]       [,4]     [,5]      [,6]
[1,] 0.09505543 0.08759649 0.05709562 0.06763294 0.281736 0.2494769


###Riesgo a nivel contrato y nivel contrato-factor de riesgo

In [0]:
%%R
#Cálculo de matriz de pérdidas y ganancias opciones

#Cálculo de VaR y CVaR por contrato
#creación de matrices de VaR, VaR delta-normal,Var delta- gamma, VaR CF por contrato
DeltaX_oir=DeltaX_port_opc[,(1:n_if_opc[1])]  
m_oir=ncol(plazos_oir)    #número de factores de riesgo
m_opc=ncol(plazos_oir)

VaRmargcDN_oir=matrix(0,1,m_opc) #Var por contrato Delta Normal
VaRmargcDG_oir=matrix(0,1,m_opc) #Var por contrato Delta Gamma Normal
VaRmargcstd_oir=matrix(0,1,m_opc) #Var por contrato estandarizado 
VaRmargcCF_oir=matrix(0,1,m_opc) #Var por contrato Cornish-Fisher
CVaRmargcDN_oir=matrix(0,1,m_opc) #CVar por contrato Delta Normal
CVaRmargcDG_oir=matrix(0,1,m_opc) #CVar por contrato Delta Gamma Normal
CVaRmargcstd_oir=matrix(0,1,m_opc) #CVar por contrato estandarizado
CVaRmargcCF_oir=matrix(0,1,m_opc) #CVar por contrato Cornish-Fisher

##factores riesgo
#Spot
VaRmargcDN_fr_oir=matrix(0,3,m_opc) #Var por factor de riesgo Delta Normal
VaRmargcDG_fr_oir=matrix(0,3,m_opc) #Var por factor de riesgo Delta Gamma
VaRmargcstd_fr_oir=matrix(0,3,m_opc) #Var por factor de riesgo estandarizado 
VaRmargcCF_fr_oir=matrix(0,3,m_opc) #Var por factor de riesgo Cornish-Fisher
CVaRmargcDN_fr_oir=matrix(0,3,m_opc) #CVar por factor de riesgo Delta Normal
CVaRmargcDG_fr_oir=matrix(0,3,m_opc) #CVar por factor de riesgo Delta Gamma
CVaRmargcstd_fr_oir=matrix(0,3,m_opc) #Var por factor de riesgo estandarizado
CVaRmargcCF_fr_oir=matrix(0,3,m_opc) #CVar por factor de riesgo Cornish-Fisher
WFR_opc=diagv(c(1,1,1))


for (i in 1:(m_opc))
{
  delta_oir=t(delta_oir_T[seq(i,m_opc*3,by=m_opc)])  #me quedo con la fila 1,6,11; o 2,7,12 etc; agrupo las columnas para formar cada contrato
  tdelta_oir=t(delta_oir_T[seq(i,m_opc*3,by=m_opc)])*xp_oir[seq(i,m_opc*3,by=m_opc)]   #delta tilde
  gamma_oir=mgammaopc(x01[i],x02[i],K_oir[i],x03[i],plazos_oir[i],cp_oir[i],cs_oir,pr_oir,dct_oir,contratos_oir[i],nominal_oir)  #paso clave
  tgamma_oir=xp_oir[seq(i,m_opc*3,by=m_opc)]%*%t(xp_oir[seq(i,m_opc*3,by=m_opc)])*gamma_oir #gamma tilde
  Varmargcaux_oir=cov(DeltaX_oir[,seq(i,m_opc*3,by=m_opc)])*((n_opc)/(n_opc-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_oir=(tgamma_oir%*%Varmargcaux_oir)
  tgammavar2_oir=(tgamma_oir%*%Varmargcaux_oir)%*%(tgamma_oir%*%Varmargcaux_oir)  
  mudg_oir=sum(diag(tgammavar_oir))/2+mu_oir[i]
  desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

  mudn_oir=ifelse(mu_oir[i]>0,0,mu_oir[i])
  desvdn_oir=sqrt(tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

  VaRmargcDN_oir[i]=mudn_oir+desvdn_oir*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
  CVaRmargcDN_oir[i]=mudn_oir-desvdn_oir*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal

  VaRmargcDG_oir[i]=mudg_oir+desv_oir*qnorm(1-alpha) #VaR Delta Gamma Normal por contrato
  CVaRmargcDG_oir[i]=mudg_oir-desv_oir*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_oir=tgammavar2_oir%*%(tgamma_oir%*%Varmargcaux_oir)
  asimPmargc_oir=sum(diag(tgammavar3_oir))+3*tdelta_oir%*%Varmargcaux_oir%*%tgamma_oir%*%Varmargcaux_oir%*%t(tdelta_oir) #tercer momento
  asimPmargc_oir=asimPmargc_oir/(desv_oir)^3  #coef de asimetría
  VaRmargcstd_oir[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_oir  #VaR estandarizado
  CVaRmargcstd_oir[i] = 0
  VaRmargcCF_oir[i]=mudg_oir+desv_oir*VaRmargcstd_oir[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_oir[i]=-mudg_oir-desv_oir*(dnorm(VaRmargcstd_oir[i])/(pnorm(VaRmargcstd_oir[i])))

#Por factor de riesgo y por contrato
#descuento, spot y volatilidad
  for (j in 1:3) #para cada contrato iteramos 3 veces
  {
    WFRM_opc=matrix(0,3,3)  #PASO CLAVE
    WFRM_opc[j,j]=1  #PASO CLAVE
    delta_oir=t(delta_oir_T[seq(i,m_opc*3,by=m_opc)])*WFR_opc[j,]
    tdelta_oir=t(delta_oir_T[seq(i,m_opc*3,by=m_opc)])*xp_oir[seq(i,m_opc*3,by=m_opc)]*WFR_opc[j,]   #delta tilde
    gamma_oir=mgammaopc(x01[i],x02[i],K_oir[i],x03[i],plazos_oir[i],cp_oir[i],cs_oir,pr_oir,dct_oir,contratos_oir[i],nominal_oir)*WFRM_opc #paso clave
   #WFR_opc[j,] son varibles  guía, para el gradiente activamos las variables que son diferentes de cero.
   #print(WFRM_opc)
    tgamma_oir=xp_oir[seq(i,m_opc*3,by=m_opc)]%*%t(xp_oir[seq(i,m_opc*3,by=m_opc)])*gamma_oir #gamma tilde
    Varmargcaux_oir=cov(DeltaX_oir[,seq(i,m_opc*3,by=m_opc)])*((n_opc)/(n_opc-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_oir=(tgamma_oir%*%Varmargcaux_oir)
    tgammavar2_oir=(tgamma_oir%*%Varmargcaux_oir)%*%(tgamma_oir%*%Varmargcaux_oir)  
    #mudg_oir=sum(diag(tgammavar_oir))/2+mu_oir[i]
    mudg_oir=0
    desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

    mudn_oir=0
    desvdn_oir=sqrt(tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

    VaRmargcDN_fr_oir[j,i]=mudn_oir+desvdn_oir*qnorm(1-alpha) #VaR por contrato-factor de riesgo Delta Normal
    CVaRmargcDN_fr_oir[j,i]=mudn_oir-desvdn_oir*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR por contrato-factor de riesgo Delta Normal

    VaRmargcDG_fr_oir[j,i]=mudg_oir+desv_oir*qnorm(1-alpha) #VaR Delta- Gamma por contrato-factor de riesgo
    CVaRmargcDG_fr_oir[j,i]=mudg_oir-desv_oir*(dnorm(qnorm(1-alpha))/(1-alpha)) #CVaR Delta- Gamma por contrato-factor de riesgo
    
    tgammavar3_oir=tgammavar2_oir%*%(tgamma_oir%*%Varmargcaux_oir)
    asimPmargc_oir=sum(diag(tgammavar3_oir))+3*tdelta_oir%*%Varmargcaux_oir%*%tgamma_oir%*%Varmargcaux_oir%*%t(tdelta_oir) #tercer momento
    asimPmargc_oir=asimPmargc_oir/(desv_oir)^3  #coef de asimetría
    VaRmargcstd_fr_oir[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_oir  #VaR estandarizado
    CVaRmargcstd_fr_oir = 0
    VaRmargcCF_fr_oir[j,i]=mudg_oir+desv_oir*VaRmargcstd_fr_oir[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_oir[j,i]=-mudg_oir-desv_oir*(dnorm(VaRmargcstd_fr_oir[j,i])/(pnorm(VaRmargcstd_fr_oir[j,i])))
  }


}

print("Valor a nivel contrato")
print(V0_oir)
print("")
print("VaR a nivel contrato Delta normal")
print(VaRmargcDN_oir)
print("")
print("VaR a nivel contrato Delta Gamma Normal")
print(VaRmargcDG_oir)
print("")
print("VaR a nivel contrato CF")
print(VaRmargcCF_oir)
print("")
print("CVaR a nivel contrato Delta normal")
print(CVaRmargcDN_oir)
print("")
print("CVaR a nivel contrato Delta Gamma Normal")
print(CVaRmargcDG_oir)
print("")
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_oir)

print("VaR a nivel contrato-factor de riesgo delta normal")
print(VaRmargcDN_fr_oir)
print("")
print("VaR a nivel contrato-factor de riesgo Delta Gamma Normal")
print(VaRmargcDG_fr_oir)
print("")
print("VaR a nivel contrato-factor de riesgo CF")
print(VaRmargcCF_fr_oir)
print("")
print("CVaR a nivel contrato-factor de riesgo delta normal")
print(CVaRmargcDN_fr_oir)
print("")
print("CVaR a nivel contrato-factor de riesgo Delta Gamma Normal")
print(CVaRmargcDG_fr_oir)
print("")
print("CVaR a nivel contrato-factor de riesgo CF")
print(CVaRmargcCF_fr_oir)



[1] "Valor a nivel contrato"
        [,1]      [,2]
[1,] 2.24108 0.1065757
[1] ""
[1] "VaR a nivel contrato Delta normal"
           [,1]        [,2]
[1,] -0.1200008 -0.03320576
[1] ""
[1] "VaR a nivel contrato Delta Gamma Normal"
         [,1]      [,2]
[1,] 10.24862 0.1052162
[1] ""
[1] "VaR a nivel contrato CF"
         [,1]      [,2]
[1,] 10.24884 0.1058008
[1] ""
[1] "CVaR a nivel contrato Delta normal"
           [,1]       [,2]
[1,] -0.1414539 -0.0391421
[1] ""
[1] "CVaR a nivel contrato Delta Gamma Normal"
         [,1]       [,2]
[1,] 10.22717 0.09927851
[1] ""
[1] "CVaR a nivel contrato CF"
          [,1]       [,2]
[1,] -10.50988 -0.1770621
[1] "VaR a nivel contrato-factor de riesgo delta normal"
            [,1]          [,2]
[1,] -0.02157008 -0.0002946282
[2,] -0.10223018 -0.0171833276
[3,] -0.04778420 -0.0296414695
[1] ""
[1] "VaR a nivel contrato-factor de riesgo Delta Gamma Normal"
            [,1]          [,2]
[1,] -0.02157020 -0.0002946284
[2,] -0.10223018 -0.0171833

###Riesgo a nivel total y total factor de riesgo

In [0]:
%%R
#RIESGO TOTAL OPCIONES TASA DE INTERÉS

VarDeltaX_oir=cov(DeltaX_oir)*(n_opc/(n_opc-1)) 

tdelta_oir= (deltaT_oir_T%*%VarDeltaX_oir)
mudn_oir=0
desvdn_oir=sqrt(tdelta_oir%*%VarDeltaX_oir%*%t(tdelta_oir))

VaRT_oir_DN=mudn_oir+desvdn_oir*qnorm(1-alpha)
CVaRT_oir_DN=mudn_oir-desvdn_oir*(dnorm(qnorm(1-alpha))/(1-alpha))

tgammavar_oir=(gammaT_oirT2%*%VarDeltaX_oir)
tgammavar2_oir=(tgammavar_oir)%*%(tgammavar_oir) 
mudg_oir=sum(diag(tgammavar_oir))/2+sum(mu_oir)
desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+deltaT_oir_T%*%VarDeltaX_oir%*%t(deltaT_oir_T))

VaRT_oir_DGM=(mudg_oir)+(desv_oir)*qnorm(1-alpha)  #VaR Gamma Normal
CVaRT_oir_DGM=(mudg_oir)-(desv_oir)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_oir=tgammavar2_oir%*%(tgammavar_oir)
asimP_oir=sum(diag(tgammavar3_oir))+3*deltaT_oir_T%*%VarDeltaX_oir%*%gammaT_oirT2%*%VarDeltaX_oir%*%t(deltaT_oir_T) #gamma tercer momento
asimP_oir=asimP_oir/(desv_oir)^3  #coef de asimetría
VaRstd_oir=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_oir  #VaR estandarizado
VaRT_oir_DGM_CF=(mudg_oir)+(desv_oir)*VaRstd_oir #VaR cf
CVaRT_oir_DGM_CF=(mudg_oir)-(desv_oir)*dnorm(VaRstd_oir)/(pnorm(VaRstd_oir))

print('VaR total Delta Normal')
print(VaRT_oir_DN)
print('CVaR total Delta Normal')
print(CVaRT_oir_DN)
print("")
print('VaR total Delta Gamma')
print(VaRT_oir_DGM)
print('CVaR total Delta Gamma')
print(CVaRT_oir_DGM)
print("")
print('VaR Cornish Fisher')
print(VaRT_oir_DGM_CF)
print('CVaR Cornish Fisher')
print(CVaRT_oir_DGM_CF)

VaRT_fr_oir_DN=matrix(0,1,3)
CVaRT_fr_oir_DN=matrix(0,1,3)
VaRT_fr_oir_DGM=matrix(0,1,3)
CVaRT_fr_oir_DGM=matrix(0,1,3)
VaRT_fr_oir_DGM_CF=matrix(0,1,3)
CVaRT_fr_oir_DGM_CF=matrix(0,1,3)

WFRT_opc=matrix(0,3,(m_opc*3))

#RIESGO TOTAL POR FACTOR DE RIESGO DE OPCIONES TASA DE INTERES

for (j in (1:3))
{
WFRT_opc[j,((j-1)*m_opc+1):(j*m_opc)]=matrix(1,1,m_opc)    #Matriz de diseño para 
WFRM_opc=matrix(0,(m_opc*3),(m_opc*3))  #PASO CLAVE
WFRM_opc[((j-1)*m_opc+1):(m_opc*j),((j-1)*m_opc+1):(m_opc*j)]=matrix(1,m_opc,m_opc)  #PASO CLAVE
gamma_oirtT2_fr=gammaT_oirT2*WFRM_opc #evaluación de gamma multiplicado por x^{2}
g_oirtT_fr=deltaT_oir_T*WFRT_opc[j,]
tgammavar_oir=(gamma_oirtT2_fr%*%VarDeltaX_oir)
tgammavar2_oir=(tgammavar_oir)%*%(tgammavar_oir) 
mudn_oir = 0
desvn_oir= sqrt(g_oirtT_fr%*%VarDeltaX_oir%*%t(g_oirtT_fr))
VaRT_fr_oir_DN[j]=(mudn_oir)+(desvn_oir)*qnorm(1-alpha) #VaR Delta Normal
CVaRT_fr_oir_DN[j]=(mudn_oir)-(desvn_oir)*dnorm(qnorm(alpha))/(1-alpha)

mudg_oir=0
desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+g_oirtT_fr%*%VarDeltaX_oir%*%t(g_oirtT_fr))
VaRT_fr_oir_DGM[j]=(mudg_oir)+(desv_oir)*qnorm(1-alpha) #VaR Gamma Normal
CVaRT_fr_oir_DGM[j]=(mudg_oir)-(desv_oir)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_oir=tgammavar2_oir%*%(tgammavar_oir)
asimP_oir=sum(diag(tgammavar3_oir))+3*g_oirtT_fr%*%VarDeltaX_oir%*%gamma_oirtT2_fr%*%VarDeltaX_oir%*%t(g_oirtT_fr) #gamma tercer momento
 #print(asimP_oir)
asimP_oir=asimP_oir/(desv_oir)^3  #coef de asimetría
 
VaRstd_oir=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_oir  #VaR estandarizado
#print(VaRstd_oir)
VaRT_fr_oir_DGM_CF[j]=(mudg_oir)+(desv_oir)*VaRstd_oir #VaR cf
CVaRT_fr_oir_DGM_CF[j]=(mudg_oir)-(desv_oir)*dnorm(VaRstd_oir)/(pnorm(VaRstd_oir))


}

print('VaR total por factor de riesgo Delta Normal')
print(VaRT_fr_oir_DN)
print('CVaR total por factor de riesgo Delta Normal')
print(CVaRT_fr_oir_DN)
print("")
print('VaR total por factor de riesgo Delta Gamma')
print(VaRT_fr_oir_DGM)
print('CVaR total por factor de riesgo Delta Gamma')
print(CVaRT_fr_oir_DGM)
print("")
print('VaR total por factor de riesgo Cornish Fisher')
print(VaRT_fr_oir_DGM_CF)
print('CVaR total por factor de riesgo Cornish Fisher')
print(CVaRT_fr_oir_DGM_CF)
print("")
print('Valoración de todas las opciones')
print(sum(V0_oir))


[1] "VaR total Delta Normal"
              [,1]
[1,] -0.0003102937
[1] "CVaR total Delta Normal"
              [,1]
[1,] -0.0003657663
[1] ""
[1] "VaR total Delta Gamma"
         [,1]
[1,] 10.37012
[1] "CVaR total Delta Gamma"
         [,1]
[1,] 10.34564
[1] ""
[1] "VaR Cornish Fisher"
         [,1]
[1,] 10.37063
[1] "CVaR Cornish Fisher"
         [,1]
[1,] 10.34609
[1] "VaR total por factor de riesgo Delta Normal"
            [,1]       [,2]        [,3]
[1,] -0.02181103 -0.1074488 -0.07171912
[1] "CVaR total por factor de riesgo Delta Normal"
            [,1]       [,2]        [,3]
[1,] -0.02571028 -0.1266578 -0.08454066
[1] ""
[1] "VaR total por factor de riesgo Delta Gamma"
            [,1]       [,2]        [,3]
[1,] -0.02181115 -0.1074488 -0.07172959
[1] "CVaR total por factor de riesgo Delta Gamma"
            [,1]       [,2]        [,3]
[1,] -0.02571042 -0.1266578 -0.08455301
[1] ""
[1] "VaR total por factor de riesgo Cornish Fisher"
            [,1]       [,2]       [,3]
[1,] -